In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pyAudioAnalysis import audioBasicIO #A
from pyAudioAnalysis import ShortTermFeatures #B
import matplotlib.pyplot as plt
import os #C

In [15]:
def preProcess( fileName ):
    [Fs, x] = audioBasicIO.read_audio_file(fileName) #A

    #B
    if( len( x.shape ) > 1 and  x.shape[1] == 2 ):
        x = np.mean( x, axis = 1, keepdims = True )
    else:
        x = x.reshape( x.shape[0], 1 )
    #C
    F, f_names = ShortTermFeatures.feature_extraction(
        x[ :, 0 ],
        Fs, 0.050*Fs,
        0.025*Fs
    )
    
    return (f_names, F)
    #return F

In [11]:
def getChromagram( audioData ):
    
    # A
    temp_data =  audioData[ 21 ].reshape( 
        1, 
        audioData[ 21 ].shape[0] 
    )
    chronograph = temp_data
    
    # B
    for i in range( 22, 33 ):
        temp_data =  audioData[ i ].reshape( 
            1, 
            audioData[ i ].shape[0] 
        )
        chronograph = np.vstack( [ chronograph,  temp_data ] )
    
    return chronograph

In [12]:
def getNoteFrequency( chromagram ):
    
    numberOfWindows = chromagram.shape[1] #A
    
    freqVal = chromagram.argmax( axis = 0 ) #B
    
    histogram, bin = np.histogram( freqVal, bins = 12 ) #C
    
    normalized_hist = histogram.reshape( 1, 12 ).astype( float ) / numberOfWindows #D
    
    return normalized_hist

In [13]:
fileList = []
def getDataset( filePath ):
    X = pd.DataFrame(  )
    
    columns=[ "G#", "G", "F#", "F", "E", "D#", "D", "C#", "C", "B", "A#", "A" ]
    
    for root, dirs, filenames in os.walk( filePath ):
        for file in filenames:
            fileList.append( file )
            feature_name, features = preProcess(filePath + file )
            chromagram = getChromagram( features )
            noteFrequency = getNoteFrequency( chromagram )
            x_new =  pd.Series(noteFrequency[ 0, : ])
            X = pd.concat( [ X, x_new ], axis = 1 )
        
    data = X.T.copy()
    data.columns = columns
    data.index = [ i for i in range( 0, data.shape[ 0 ] ) ]
            
    return data

In [16]:
preProcess("C:/Users/hlynn/Documents/Deep-learning_jolpu_lab/wav/ArianaGrande-7rings(1).wav")

(['zcr',
  'energy',
  'energy_entropy',
  'spectral_centroid',
  'spectral_spread',
  'spectral_entropy',
  'spectral_flux',
  'spectral_rolloff',
  'mfcc_1',
  'mfcc_2',
  'mfcc_3',
  'mfcc_4',
  'mfcc_5',
  'mfcc_6',
  'mfcc_7',
  'mfcc_8',
  'mfcc_9',
  'mfcc_10',
  'mfcc_11',
  'mfcc_12',
  'mfcc_13',
  'chroma_1',
  'chroma_2',
  'chroma_3',
  'chroma_4',
  'chroma_5',
  'chroma_6',
  'chroma_7',
  'chroma_8',
  'chroma_9',
  'chroma_10',
  'chroma_11',
  'chroma_12',
  'chroma_std',
  'delta zcr',
  'delta energy',
  'delta energy_entropy',
  'delta spectral_centroid',
  'delta spectral_spread',
  'delta spectral_entropy',
  'delta spectral_flux',
  'delta spectral_rolloff',
  'delta mfcc_1',
  'delta mfcc_2',
  'delta mfcc_3',
  'delta mfcc_4',
  'delta mfcc_5',
  'delta mfcc_6',
  'delta mfcc_7',
  'delta mfcc_8',
  'delta mfcc_9',
  'delta mfcc_10',
  'delta mfcc_11',
  'delta mfcc_12',
  'delta mfcc_13',
  'delta chroma_1',
  'delta chroma_2',
  'delta chroma_3',
  'delta ch

In [47]:
data = getDataset("C:/Users/hlynn/Documents/Deep-learning_jolpu_lab/wav/")

In [48]:
data

,G#,G,F#,F,E,D#,D,C#,C,B,A#,A
0,0.443038,0.0,0.0,0.000000,0.000000,0.0,0.518987,0.0,0.000000,0.000000,0.0,0.037975
1,0.126582,0.0,0.0,0.012658,0.000000,0.0,0.303797,0.0,0.000000,0.544304,0.0,0.012658
2,0.037975,0.0,0.0,0.000000,0.392405,0.0,0.000000,0.0,0.531646,0.000000,0.0,0.037975
3,0.164557,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.835443
4,0.291139,0.0,0.0,0.000000,0.088608,0.0,0.582278,0.0,0.000000,0.000000,0.0,0.037975


In [70]:
k = 3
epochs = 1000

In [71]:
def initilizeCentroids( data, k ):

    centroids = data[ 0: k ] #A
    
    return centroids

In [72]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder( dtype = tf.float32 )
C = tf.compat.v1.placeholder( dtype = tf.float32 )
C_labels = tf.compat.v1.placeholder( dtype = tf.int32 )

In [73]:
# utility to assign centroids to examples
expanded_vectors = tf.expand_dims( X, 0 )
expanded_centroids = tf.expand_dims( C, 1 )
distance = tf.reduce_sum( tf.square( tf.subtract( expanded_vectors, expanded_centroids ) ), axis = 2 )
getCentroidsOp = tf.argmin( distance, 0 )


# utility to recalculate centroids
sums = tf.math.unsorted_segment_sum( X, C_labels, k )
counts = tf.math.unsorted_segment_sum( tf.ones_like( X ), C_labels, k  )
reCalculateCentroidsOp = tf.math.divide( sums,  counts )

In [74]:
"""def assignCentroids(X, C):  
    expanded_vectors = tf.expand_dims(X, 0)
    expanded_centroids = tf.expand_dims(C, 1)
    distance = tf.math.reduce_sum( tf.math.square( tf.math.subtract( expanded_vectors, expanded_centroids ) ), axis=2 )
    return tf.math.argmin(distance, 0)
                                              
# utility to recalculate centroids
def reCalculateCentroids(X, X_labels):
    sums = tf.math.unsorted_segment_sum( X, X_labels, k )
    counts = tf.math.unsorted_segment_sum( tf.ones_like( X ), X_labels, k  )
    return tf.math.divide( sums, counts )  """

'def assignCentroids(X, C):  \n    expanded_vectors = tf.expand_dims(X, 0)\n    expanded_centroids = tf.expand_dims(C, 1)\n    distance = tf.math.reduce_sum( tf.math.square( tf.math.subtract( expanded_vectors, expanded_centroids ) ), axis=2 )\n    return tf.math.argmin(distance, 0)\n                                              \n# utility to recalculate centroids\ndef reCalculateCentroids(X, X_labels):\n    sums = tf.math.unsorted_segment_sum( X, X_labels, k )\n    counts = tf.math.unsorted_segment_sum( tf.ones_like( X ), X_labels, k  )\n    return tf.math.divide( sums, counts )  '

In [75]:
centroids = []
data_labels = []

with tf.compat.v1.Session() as sess:

    # Initilize all tensor flow variables
    sess.run( tf.compat.v1.global_variables_initializer() )

    # Get initial list of k centroids
    centroids = initilizeCentroids( data, k )

    for epoch in range( epochs ):
        data_labels =  sess.run( getCentroidsOp, feed_dict = { X: data, C: centroids } )
        centroids = sess.run( reCalculateCentroidsOp, feed_dict = { X: data, C_labels: data_labels } )

print( data_labels )
print( centroids )

[0 1 2 0 0]
[[0.29957807 0.         0.         0.         0.02953587 0.
  0.36708865 0.         0.         0.         0.         0.30379745]
 [0.12658228 0.         0.         0.01265823 0.         0.
  0.30379745 0.         0.         0.5443038  0.         0.01265823]
 [0.03797468 0.         0.         0.         0.39240506 0.
  0.         0.         0.5316456  0.         0.         0.03797468]]


In [76]:
final_labels = pd.DataFrame( { "Labels": data_labels, "File Names": fileList } )
final_labels

,Labels,File Names
0,0,ArianaGrande-7rings(1).wav
1,1,ArianaGrande-7rings(2).wav
2,2,ArianaGrande-7rings(3).wav
3,0,The Chainsmokers & Coldplay - Something Just L...
4,0,The Chainsmokers & Coldplay - Something Just L...
